In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
%cd stylegan2-ada-pytorch/

In [ ]:
%ls

In [4]:
!mkdir results

In [ ]:
!pip3 install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [6]:
!python dataset_tool.py --help

Usage: dataset_tool.py [OPTIONS]

  Convert an image dataset into a dataset archive usable with StyleGAN2 ADA
  PyTorch.

  The input dataset format is guessed from the --source argument:

  --source *_lmdb/                    Load LSUN dataset
  --source cifar-10-python.tar.gz     Load CIFAR-10 dataset
  --source train-images-idx3-ubyte.gz Load MNIST dataset
  --source path/                      Recursively load all images from path/
  --source dataset.zip                Recursively load all images from dataset.zip

  Specifying the output format and path:

  --dest /path/to/dir                 Save output files under /path/to/dir
  --dest /path/to/dataset.zip         Save output files into /path/to/dataset.zip

  The output dataset format can be either an image folder or an uncompressed
  zip archive. Zip archives makes it easier to move datasets around file
  servers and clusters, and may offer better training performance on network
  file systems.

  Images within the dataset archi

In [ ]:
!python dataset_tool.py --source="/kaggle/input/batik-dataset-for-gan/Dataset Final" --dest=./datasets/dataset.zip --width=128 --height=128

In [8]:
%ls

Dockerfile*       dataset_tool.py*  docs/         projector.py*     train.py*
LICENSE.txt*      datasets/         generate.py*  results/          training/
README.md*        dnnlib/           legacy.py*    style_mixing.py*
calc_metrics.py*  docker_run.sh*    metrics/      torch_utils/


In [9]:
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in the paper "Training
  Generative Adversarial Networks with Limited Data".

  Examples:

  # Train with custom dataset using 1 GPU.
  python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1

  # Train class-conditional CIFAR-10 using 2 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/cifar10.zip \
      --gpus=2 --cfg=cifar --cond=1

  # Transfer learn MetFaces from FFHQ using 4 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/metfaces.zip \
      --gpus=4 --cfg=paper1024 --mirror=1 --resume=ffhq1024 --snap=10

  # Reproduce original StyleGAN2 config F.
  python train.py --outdir=~/training-runs --data=~/datasets/ffhq.zip \
      --gpus=8 --cfg=stylegan2 --mirror=1 --aug=noaug

  Base configs (--cfg):
    auto       Automatically select reasonable defaults based on resolution
               and GPU count. Good starting point for new datasets.
    stylegan2  Rep

In [ ]:
!python train.py --outdir=./results --data=./datasets/dataset.zip --dry-run


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./datasets/dataset.zip",
    "use_labels": false,
    "max_size": 5330,
    "xflip": false,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512,

In [ ]:
!python train.py --outdir=./results --data=./datasets/dataset.zip --snap=5 --kimg=1000